In [226]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [227]:
df = pd.read_csv("AB_NYC_2019.csv")

In [228]:
df_hw = df[['neighbourhood_group','room_type','latitude','longitude','price','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']]

In [229]:
# Q 1
df_hw['neighbourhood_group'].mode()
#Mode is Manhattan

0    Manhattan
dtype: object

In [230]:
above_average=[]
for i in range(0,len(df_hw)):
    if df_hw['price'][i]>= 152:
        above_average.append(1)
    else:
        above_average.append(0)
df_hw['above_average']= above_average

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [231]:
df_hw.head()

,neighbourhood_group,room_type,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,above_average
0,Brooklyn,Private room,40.64749,-73.97237,149,1,9,0.21,6,365,0
1,Manhattan,Entire home/apt,40.75362,-73.98377,225,1,45,0.38,2,355,1
2,Manhattan,Private room,40.80902,-73.94190,150,3,0,NaN,1,365,0
3,Brooklyn,Entire home/apt,40.68514,-73.95976,89,1,270,4.64,1,194,0
4,Manhattan,Entire home/apt,40.79851,-73.94399,80,10,9,0.10,1,0,0


In [232]:
#df_hw['neighbourhood_group'].value_counts()

In [233]:
from sklearn.model_selection import train_test_split

In [234]:
df_full_train, df_test = train_test_split(df_hw, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [235]:
df_train =df_train.reset_index(drop=True)
df_val =df_val.reset_index(drop=True)
df_test =df_test.reset_index(drop=True)
y_train = np.nan_to_num(df_train.price.values)
y_val = np.nan_to_num(df_val.price.values)
y_test = np.nan_to_num(df_test.price.values)
y_train_ridge = np.log1p(df_train.price.values)
del df_train['price']
del df_val['price']
del df_test['price']

In [236]:
df_full_train['reviews_per_month'] = df_full_train['reviews_per_month'].fillna(0)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [237]:
df_numerical = df_train[['latitude','longitude','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']]

In [238]:
corrM = df_numerical.corr()

In [239]:
#Q 2
corrM
#number_of_reviews and reviews_per_month have highest correlation of 0.549792

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,0.002646,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.140201,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.111121,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.549792,-0.073167,0.174477
reviews_per_month,0.002646,0.140201,-0.111121,0.549792,1.000000,-0.007055,0.188765
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.007055,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.188765,0.225913,1.000000


In [240]:
df_categorical = df_train[['neighbourhood_group','room_type']]

In [241]:
from sklearn.metrics import mutual_info_score
#df_categorical['neighbourhood_group'].count(),df_train['price'].count()
mi = mutual_info_score(df_categorical['neighbourhood_group'],df_train['above_average'])
mi_2 = mutual_info_score(df_categorical['room_type'],df_train['above_average'])

In [242]:
#Q 3
round(mi, 2),round(mi_2, 2)
#room_type has a higher mutual information score

(0.05, 0.14)

In [243]:
numerical = ['latitude','longitude','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']
categorical = ['neighbourhood_group','room_type']

In [244]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [245]:
#X_train.isnull().sum()
#array_has_nan = np. isnan(X_train).sum()
#print(array_has_nan)
X_train = np.nan_to_num(X_train)
X_val = np.nan_to_num(X_val)

In [246]:
def logistic_regression(xi):
    score = w0
    
    for j in range(len(w)):
        score = score + xi[j] * w[j]
        
    result = sigmoid(score)
    return result

In [247]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=42)

In [248]:
model.intercept_[0]

0.00014222326423096022

In [249]:
#Q 4
model.coef_[0].round(2)

array([-0.  , -0.01,  0.01, -0.01,  0.01,  0.  ,  0.  , -0.  , -0.  ,
       -0.  ,  0.02,  0.  , -0.  ,  0.  ,  0.  ])

In [250]:
y_pred

array([0.00029532, 0.00025794, 0.00034261, ..., 0.0002716 , 0.00026713,
       0.00027352])

In [251]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.above_average)

In [252]:
#Q 5
#df_full_train_all = np.nan_to_num(df_full_train[categorical+numerical])
mi = df_full_train[categorical + numerical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

/usr/local/lib/python3.7/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)


latitude                          0.295236
longitude                         0.248568
room_type                         0.142390
neighbourhood_group               0.046223
calculated_host_listings_count    0.036605
reviews_per_month                 0.016342
minimum_nights                    0.013495
availability_365                  0.011640
number_of_reviews                 0.009806
dtype: float64

In [253]:
from sklearn.linear_model import Ridge

In [254]:
#Q6
from sklearn.model_selection import cross_val_score
from numpy import absolute
from numpy import mean
from numpy import std
clf = Ridge(alpha=1.0)
alpha_val = [0, 0.01, 0.1, 1, 10]
for i in range(0,len(alpha_val)):
    clf = Ridge(alpha=alpha_val[i])
    scores = cross_val_score(clf, X_train, y_train_ridge, scoring='neg_root_mean_squared_error', n_jobs=-1)
    print (float(alpha_val[i]),round(mean(scores),8))
    #print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))
#OP = clf.fit(X_train, y_train_ridge)

# force scores to be positive
#scores = absolute(scores)


0.0 -0.50164349
0.01 -0.50153881
0.1 -0.50153886
1.0 -0.5015562
10.0 -0.50244524


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=6.96525e-21): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=5.84755e-22): result may not be accurate.
  overwrite_a=True).T
